In [4]:
import os
import sys
sys.path.append(os.path.abspath('../../../..'))

import numpy as np
import pandas as pd

# from utlis.corr_utlis.processed_syned_load import load_filtered_data_from_h5

In [7]:
rec_path = '/data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_25/20241002PMCr2_17_05'

hdf5_file_path = os.path.join(rec_path, 'MIR_Aligned/aligned_predictions_with_ca_and_dF_F.h5')

# Load the DataFrame from the HDF5 file
df_merged_with_dF_F = pd.read_hdf(hdf5_file_path, key='df')

In [10]:
# List of keypoints to analyze (indices)
head_keypoints = [1, 2, 3, 4]  # Example: EarL, EarR, Snout, SpineF
# z_columns = [f'kp{idx}_z' for idx in keypoints_of_interest]
head_coords = df_merged_with_dF_F[[f"kp{idx}_{axis}" for idx in head_keypoints for axis in ['x', 'y', 'z']]]



In [ ]:
# import numpy as np
# import pandas as pd

# # Modified pairwise_distances function
# def pairwise_distances_from_df(df, keypoints):
#     results = []
#     for _, row in df.iterrows():
#         coords = []
#         # Extract coordinates for each specified keypoint
#         for idx in keypoints:
#             coords.append(np.array([row[f'kp{idx}_x'], row[f'kp{idx}_y'], row[f'kp{idx}_z']]))
        
#         # Calculate pairwise distances
#         distances = [
#             np.linalg.norm(coords[i] - coords[j])
#             for i in range(len(coords))
#             for j in range(i + 1, len(coords))
#         ]
#         results.append(np.mean(distances))  # Store the mean of distances
#     return results

# mean_head = pairwise_distances_from_df(head_coords, head_keypoints)

In [22]:
import numpy as np

head_keypoints = ['kp1', 'kp2', 'kp3', 'kp4']  # Adjust based on head keypoints
head_coords = df_merged_with_dF_F[[f"{kp}_{axis}" for kp in head_keypoints for axis in ['x', 'y', 'z']]]

def pairwise_distances(coords):
    distances = []
    for i in range(len(coords)):
        for j in range(i + 1, len(coords)):
            dist = np.linalg.norm(coords[i] - coords[j])
            distances.append(dist)
    return np.mean(distances)

head_geometry = head_coords.apply(lambda row: pairwise_distances(row.values.reshape(-1, 3)), axis=1)


In [23]:
head_geometry

timestamp_ms_mini
-28        30.062938
 76        30.299447
 173       30.601935
 284       30.039744
 383       30.205693
             ...    
 895154    27.088338
 895262    27.757364
 895354    27.380261
 895459    27.861828
 895566    27.066504
Length: 8894, dtype: float64

In [24]:
avg_geometry = head_geometry.mean()
template_time = abs(head_geometry - avg_geometry).idxmin()
template_coords = head_coords.loc[template_time].values.reshape(-1, 3)


In [25]:
def calculate_centroid(row):
    return np.mean(row.values.reshape(-1, 3), axis=0)

centroids = head_coords.apply(calculate_centroid, axis=1)


In [26]:
from scipy.spatial.transform import Rotation as R

def align_template(frame_coords, template_coords):
    centroid_frame = np.mean(frame_coords, axis=0)
    centroid_template = np.mean(template_coords, axis=0)
    
    centered_frame = frame_coords - centroid_frame
    centered_template = template_coords - centroid_template
    
    rotation, _, _ = np.linalg.svd(centered_frame.T @ centered_template)
    aligned_coords = rotation @ centered_frame.T + centroid_template.T
    return aligned_coords.T

aligned_coords = head_coords.apply(
    lambda row: align_template(row.values.reshape(-1, 3), template_coords), axis=1
)


ValueError: operands could not be broadcast together with shapes (3,4) (3,) 